# Working with h5ad files

This notebook will discuss how cells are linked to the cell-by-gene data stored in .h5ad files. It will also discuss how to work with .h5ad files using the anndata library.

In [1]:
import anndata
import h5py
import numpy as np
import os
import pandas as pd
import pathlib
import scipy.sparse

In [2]:
import abc_atlas_access.abc_atlas_cache.abc_project_cache as abc_project_cache

In [3]:
cache_dir = "/Users/scott.daniel/KnowledgeEngineering/cell_type_mapper/examples/data/abc_cache"

if not os.path.isdir(cache_dir):
    raise RuntimeError(
        "Set cache_dir above to the path to the directory where you want to download ABC Atlas data"
    )

abc_cache = abc_project_cache.AbcProjectCache.from_cache_dir(cache_dir)

/Users/scott.daniel/miniconda3/envs/webinar_2026_02/lib/python3.12/site-packages/abc_atlas_access/abc_atlas_cache/cloud_cache.py:679: OutdatedManifestWarning: You are loading releases/20250531/manifest.json. A more up to date version of the dataset -- releases/20251031/manifest.json -- exists online. To see the changes between the two versions of the dataset, run
type.compare_manifests('releases/20250531/manifest.json', 'releases/20251031/manifest.json')
To load another version of the dataset, run
type.load_manifest('releases/20251031/manifest.json')
  warnings.warn(msg, OutdatedManifestWarning)


# 1. What are h5ad files?

h5ad files are how the actual cell-by-gene expression data is distributed in `abc_atlas_access`. They are a file format designed to work with the Python library anndata.

anndata is a Python library and framework for manipulating data and metadata simultaneously (the "ann" stands for "annotated"). [Their official documentation is here](https://anndata.readthedocs.io/en/stable/). h5ad is a file format, based on HDF5, for serializing and distributing anndata objects. All of the cell-by-gene data in `abc_atlas_access` is distributed as .h5ad files. h5ad files are [documented here](https://anndata.readthedocs.io/en/stable/fileformat-prose.html).

## Create cartoon data for examples

Please just run the cells in this section without worrying about what they do. The actual data in `abc_atlas_access` is very large. In order to demonstrate some of the subtleties involved in working with `h5ad` files without crashing your computer, we are going to generate some smaller cartoon data here.

In [4]:
cartoon_data_dir = pathlib.Path('cartoon_data')

if not cartoon_data_dir.exists():
    cartoon_data_dir.mkdir()

if not cartoon_data_dir.is_dir():
    raise RuntimeError(f'{cartoon_data_dir} is not a dir')

In [5]:
n_cells = 300000
n_genes = 5000

In [6]:
def create_data(dst_dir, n_cells, n_genes):
    csr_path = dst_dir / 'as_csr.h5ad'
    csc_path = dst_dir / 'as_csc.h5ad'
    dense_path = dst_dir / 'as_dense.h5ad'

    rng = np.random.default_rng(6713121)

    obs = pd.DataFrame(
        [{'cell': f'cell_{ii}', 'donor': f'donor_{rng.integers(0, 3)}'}
         for ii in range(n_cells)]
    ).set_index('cell')

    var = pd.DataFrame(
        [{'gene_id': f'ENS{ii:07d}', 'symbol': f'sym{ii}'}
         for ii in range(n_genes)]
    ).set_index('gene_id')


    data = np.zeros(n_cells*n_genes, dtype=np.int32)
    chunk_size = n_cells*n_genes//5
    for i0 in range(0, n_cells*n_genes, chunk_size):
        i1 = min(n_cells*n_genes, i0+chunk_size)
        chosen_idx = rng.choice(np.arange(i0, i1, 1), (i1-i0)//5, replace=False)
        data[chosen_idx] = rng.integers(1, 999, len(chosen_idx)).astype(np.int32)
    data = data.reshape((n_cells, n_genes))
    print('created data')

    as_dense = anndata.AnnData(obs=obs, var=var, X=data)
    as_dense.write_h5ad(dense_path)
    del as_dense
    print(f'wrote {dense_path}')

    as_csc = anndata.AnnData(obs=obs, var=var, X=scipy.sparse.csc_matrix(data))
    as_csc.write_h5ad(csc_path)
    del as_csc
    print(f'wrote {csc_path}')

    as_csr = anndata.AnnData(obs=obs, var=var, X=scipy.sparse.csr_matrix(data))
    as_csr.write_h5ad(csr_path)
    del as_csr
    print(f'wrote {csr_path}')

    return {
        'dense': dense_path,
        'csc_path': csc_path,
        'csr_path': csr_path
    }



In [7]:
path_lookup = create_data(
    dst_dir=cartoon_data_dir,
    n_cells=n_cells,
    n_genes=n_genes
)

dense_path = path_lookup['dense']
csc_path = path_lookup['csc_path']
csr_path = path_lookup['csr_path']

created data
wrote cartoon_data/as_dense.h5ad
wrote cartoon_data/as_csc.h5ad
wrote cartoon_data/as_csr.h5ad


## Anatomy of h5ad files

h5ad files are serializations of `AnnData` objects. `AnnData` objects are designed to hold cell-by-gene expression data alongside a lot of optional metadata. The h5ad files distributed by `abc_atlas_access` are relatively stripped down. We do not exercise all of the options that the anndata library makes available. In this notebook, we will discuss the features of these stripped down `AnnData` objects. To see all of the features available, consult [the official anndata documentation](https://anndata.readthedocs.io/en/stable/).

First, let us open one of our example h5ad files.

**Note:** when dealing with real (large) h5ad files, the `backed='r'` kwarg is very important. Without that, anndata will try to read all of the data in the h5ad file into memory at once, which could be prohibitively large. Reading the file with `backed='r'` as below gives you the option to only read data into memory when you need it.

In [8]:
h5ad_example = anndata.read_h5ad(dense_path, backed='r')

`AnnData` objects are arranged around arrays referred to as `X` in which rows are cells and columns are genes.

In [9]:
h5ad_example.X

<HDF5 dataset "X": shape (300000, 5000), type "<i4">

Here we have a dataset with 300000 cells and 5000 genes (which, consulting the cells in which we created this cartoon data, is what we expected).

The `AnnData` object also carries with it two pandas DataFrames.
- `obs` contains all of the metadata related to the cells in the dataset
- `var` contains all of the metadata related to the genes in the dataset

In [10]:
h5ad_example.obs

,donor
cell,
cell_0,donor_1
cell_1,donor_1
cell_2,donor_1
cell_3,donor_0
cell_4,donor_1
...,...
cell_299995,donor_0
cell_299996,donor_0
cell_299997,donor_1


In [11]:
h5ad_example.var

,symbol
gene_id,
ENS0000000,sym0
ENS0000001,sym1
ENS0000002,sym2
ENS0000003,sym3
ENS0000004,sym4
...,...
ENS0004995,sym4995
ENS0004996,sym4996
ENS0004997,sym4997


To get `X` as a numpy array, call `to_memory()`

In [12]:
h5ad_example = h5ad_example.to_memory()

In [13]:
h5ad_example.X

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [759,   0,   0, ...,   0,   0, 644],
       [  0,   0,   0, ...,   0,   0,   0],
       [712, 947,   0, ...,   0,   0, 853]],
      shape=(300000, 5000), dtype=int32)

`X` is now a numpy array that can be manipulated like any other numpy array

In [14]:
h5ad_example.X.sum(axis=1)

array([491524, 467316, 526166, ..., 490179, 488071, 524453],
      shape=(300000,))

In [15]:
h5ad_example.X.mean(axis=0)

array([ 99.50587   , 100.44607333,  99.50548   , ...,  99.70915   ,
        99.82104333, 100.55265   ], shape=(5000,))

**Note:** the anndata library is not always as careful as we would like about closing file handles. When deleting an `AnnData` object you have read from disk, it behoves you to call `anndata_obj.file.close()` to make sure the file you read it from is not left open.

In [16]:
h5ad_example.file.close()
del h5ad_example

Often, the cell-by-gene data stored in the `AnnData` object is very sparse. It is efficient, in these cases, to store the data as a sparse array. This is what we did in the `csr_path` and `csc_path` files above.

In [17]:
h5ad_example = anndata.read_h5ad(csr_path, backed='r')

In [18]:
h5ad_example.X

CSRDataset: backend hdf5, shape (300000, 5000), data_dtype int32

In [19]:
h5ad_example = h5ad_example.to_memory()

In [20]:
h5ad_example.X

<Compressed Sparse Row sparse matrix of dtype 'int32'
	with 300000000 stored elements and shape (300000, 5000)>

In this case, `X` is stored as a sparse matrix as defined by [the scipy.sparse module](https://docs.scipy.org/doc/scipy/reference/sparse.html). Many numerical operations that can be performed on numpy arrays can also be performed on these sparse matrices (albeit somewhat slower).

In [21]:
%%time
h5ad_example.X.sum(axis=1)

CPU times: user 201 ms, sys: 797 ms, total: 997 ms
Wall time: 1.05 s


matrix([[491524],
        [467316],
        [526166],
        ...,
        [490179],
        [488071],
        [524453]], shape=(300000, 1))

In [22]:
%%time
h5ad_example.X.mean(axis=0)

CPU times: user 711 ms, sys: 1.41 s, total: 2.12 s
Wall time: 2.3 s


matrix([[ 99.50587   , 100.44607333,  99.50548   , ...,  99.70915   ,
          99.82104333, 100.55265   ]], shape=(1, 5000))

To convert the arrays into dense (with all of the zeros included) numpy arrays, call `toarray()` (this will, of course, incur a memory penalty as a lot more data will be stored in RAM).

In [23]:
dense_x = h5ad_example.X.toarray()

In [24]:
%%time
dense_x.sum(axis=1)

CPU times: user 288 ms, sys: 19.5 ms, total: 308 ms
Wall time: 307 ms


array([491524, 467316, 526166, ..., 490179, 488071, 524453],
      shape=(300000,))

In [25]:
%%time
dense_x.mean(axis=0)

CPU times: user 643 ms, sys: 21.3 ms, total: 665 ms
Wall time: 667 ms


array([ 99.50587   , 100.44607333,  99.50548   , ...,  99.70915   ,
        99.82104333, 100.55265   ], shape=(5000,))

In [26]:
del dense_x
h5ad_example.file.close()
del h5ad_example

## Slicing h5ad files

Often, users want to subset cell-by-gene data either by cells or by genes. In this section, we discuss now to perform this operation for `AnnData` objects, and what the flavor of storage (dense versus sparse) means about various slicing operations.

There are two flavors of sparse matrix that anndata supports
- CSR ("Compressed Sparse Row") matrices are stored in such a way that they are optimized for slicing by rows (cells)
- CSC ("Compressed Sparse Column") matrices are stored in such a way that they are optimized for slicing by columns (genes)

In the cells above, we wrote the same cell-by-gene data three ways:
- `dense` data is stored as an (n_cells, n_genes) array with all elements, (even those that are zero) saved to disk
- `csr` data is stored in "Compressed Sparse Row" format, which is optimized for slicing on rows (cells)
- `csc` data is stored in "Compressed Sparse Column" format, which is optimized for slcing on columns (genes)

In [27]:
for file_path in (dense_path, csr_path, csc_path):
    gb = os.stat(file_path).st_size / (1024**3)
    print(f'{file_path.name} is {gb:.2e} GB in size')

as_dense.h5ad is 5.60e+00 GB in size
as_csr.h5ad is 2.25e+00 GB in size
as_csc.h5ad is 2.25e+00 GB in size


In [28]:
dense = anndata.read_h5ad(dense_path, backed='r')
csc = anndata.read_h5ad(csc_path, backed='r')
csr = anndata.read_h5ad(csr_path, backed='r')

Not the different representations of `X`.

In [29]:
dense.X

<HDF5 dataset "X": shape (300000, 5000), type "<i4">

In [30]:
csc.X

CSCDataset: backend hdf5, shape (300000, 5000), data_dtype int32

In [31]:
csr.X

CSRDataset: backend hdf5, shape (300000, 5000), data_dtype int32

### Slicing by cells

Ultimately, you can slice an anndata object the same way you would slice a numpy array. For instance, to grab a specific set of cells denoted by the indices (i.e. their row numbers) you could just:

In [32]:
cell_idx = np.array([0, 3, 5, 290000, 290001, 290002])

In [33]:
subset_of_cells = dense[cell_idx, :]

You now have an `AnnData` object sliced to include only the cells specified by `cell_idx`. Note that the slicing is automatically applied to both `X` and `obs`. `var` is unaffected since we did not slice on genes.

In [34]:
subset_of_cells.X

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0, 575,   0, ...,   0,   0, 501],
       [899,   0,   0, ...,   0,   0,  70],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ..., 330,   0,   0],
       [  0, 821,   0, ...,   0,   0,   0]], shape=(6, 5000), dtype=int32)

In [35]:
subset_of_cells.obs

,donor
cell,
cell_0,donor_1
cell_3,donor_0
cell_5,donor_1
cell_290000,donor_1
cell_290001,donor_0
cell_290002,donor_2


In [36]:
subset_of_cells.var

,symbol
gene_id,
ENS0000000,sym0
ENS0000001,sym1
ENS0000002,sym2
ENS0000003,sym3
ENS0000004,sym4
...,...
ENS0004995,sym4995
ENS0004996,sym4996
ENS0004997,sym4997


However, slicing the three `AnnData` objects (which contain nominally identical data) along the cell axis will take very different resources.

Let us subset our three `AnnData` objects along the cell axis, using Jupyter's `%%time` magic function to see how long each operation takes.

**Note:** the `to_memory()` call is necessary to tell `anndata` to actually load the subset of data into memory. Otherwise, `anndata` will just construct a pointer to the subset of cells you need, and everything will appear lightning fast until you actually try to process the subsetted data.

In [37]:
%%time
from_dense = dense[cell_idx, :].to_memory()

CPU times: user 1.82 ms, sys: 763 μs, total: 2.58 ms
Wall time: 1.96 ms


In [38]:
%%time
from_csr = csr[cell_idx, :].to_memory()

CPU times: user 2.04 ms, sys: 1.49 ms, total: 3.53 ms
Wall time: 2.81 ms


In [39]:
%%time
from_csc = csc[cell_idx, :].to_memory()

CPU times: user 329 ms, sys: 457 ms, total: 786 ms
Wall time: 1.06 s


Note how much longer it took to subset the CSC `AnnData` object than either the dense or the CSR `AnnData` object. Nevertheless, the three operations produced the same output.

In [40]:
np.testing.assert_array_equal(from_dense.X, from_csc.X.toarray())

In [41]:
np.testing.assert_array_equal(from_dense.X, from_csr.X.toarray())

To see why this is the case, let's call the convenience function `read_and_profile.read` defined in the `read_and_profile.py` code that came with this notebook. This function spins up a child process to subset the h5ad file, using the parent process to keep track of the maximum amount of memory used by the reading process.

In [42]:
import read_and_profile

In [43]:
read_and_profile.read(
    path=dense_path,
    axis='cell',
    idx=cell_idx
)

slicing as_dense.h5ad along 'cell'
took 2.49e+00 ms
maximum memory footprint 1.58e-01 GB (file size is 5.60e+00 GB)


In [44]:
read_and_profile.read(
    path=csr_path,
    axis='cell',
    idx=cell_idx
)

slicing as_csr.h5ad along 'cell'
took 3.45e+00 ms
maximum memory footprint 1.58e-01 GB (file size is 2.25e+00 GB)


In [45]:
read_and_profile.read(
    path=csc_path,
    axis='cell',
    idx=cell_idx
)

slicing as_csc.h5ad along 'cell'
took 7.18e+02 ms
maximum memory footprint 2.44e+00 GB (file size is 2.25e+00 GB)


The salient feature to notice is that the memory footprint of the processes which subset the dense and CSR matrices along the cell axis are significantly smaller than the size of the file. However, in order to subset the CSC matrix along the cell axis, the entire file had to be loaded into memory. We see something similar when we slice along the gene axis: the dense and CSC matrices can be efficiently sliced along the gene axis. The CSR matrix cannot.

## Slicing by genes

If we subset by genes, we will see that the opposite is true. Subsetting the CSR `AnnData` object will take much longer than subsetting either the dense or the CSC `AnnData` object since the CSR `AnnData` object is optimized for subsetting on cells.

In [46]:
gene_idx = np.array([5, 6, 7, 1501, 1502, 1503])

In [47]:
subset_of_genes = dense[:, gene_idx]

In [48]:
subset_of_genes.X

array([[  0, 524,   0,   0,   0,   0],
       [  0,   0, 679,   0,   0,   0],
       [  0, 907,   0,   0,  70,   0],
       ...,
       [  0,   0, 192,   0,   0,   0],
       [  0,   0,   0, 318,   0,   0],
       [  0,   0,   0,   0,   0, 370]], shape=(300000, 6), dtype=int32)

In [49]:
subset_of_genes.obs

,donor
cell,
cell_0,donor_1
cell_1,donor_1
cell_2,donor_1
cell_3,donor_0
cell_4,donor_1
...,...
cell_299995,donor_0
cell_299996,donor_0
cell_299997,donor_1


In [50]:
subset_of_genes.var

,symbol
gene_id,
ENS0000005,sym5
ENS0000006,sym6
ENS0000007,sym7
ENS0001501,sym1501
ENS0001502,sym1502
ENS0001503,sym1503


In [51]:
%%time
from_dense = dense[:, gene_idx].to_memory()

CPU times: user 42.5 ms, sys: 432 ms, total: 475 ms
Wall time: 475 ms


In [52]:
%%time
from_csr = csr[:, gene_idx].to_memory()

CPU times: user 361 ms, sys: 513 ms, total: 874 ms
Wall time: 1.19 s


In [53]:
%%time
from_csc = csc[:, gene_idx].to_memory()

CPU times: user 28.4 ms, sys: 6.14 ms, total: 34.5 ms
Wall time: 35.8 ms


In [54]:
np.testing.assert_array_equal(from_dense.X, from_csc.X.toarray())

In [55]:
np.testing.assert_array_equal(from_dense.X, from_csr.X.toarray())

In [56]:
read_and_profile.read(
    path=dense_path,
    axis='gene',
    idx=gene_idx
)

slicing as_dense.h5ad along 'gene'
took 5.30e+02 ms
maximum memory footprint 1.72e-01 GB (file size is 5.60e+00 GB)


In [57]:
read_and_profile.read(
    path=csr_path,
    axis='gene',
    idx=gene_idx
)

slicing as_csr.h5ad along 'gene'
took 1.12e+03 ms
maximum memory footprint 2.43e+00 GB (file size is 2.25e+00 GB)


In [58]:
read_and_profile.read(
    path=csc_path,
    axis='gene',
    idx=gene_idx
)

slicing as_csc.h5ad along 'gene'
took 3.16e+01 ms
maximum memory footprint 1.70e-01 GB (file size is 2.25e+00 GB)


Again, subsetting the CSR `AnnData` object along its sub-optimal axis involves reading the entire file into memory. This could be a problem when dealing with actual cell-by-gene data, which tends to be quite large.

### Takeaway

Before deciding how to manipulate an AnnData object, check to see how the data is stored on disk. You can probably infer this just by printing the result of `my_anndata.X` to stdout, i.e.

In [59]:
print(dense_path.name, '--', dense.X)
print(csr_path.name, '--', csr.X)
print(csr_path.name, '--', csc.X)

as_dense.h5ad -- <HDF5 dataset "X": shape (300000, 5000), type "<i4">
as_csr.h5ad -- CSRDataset: backend hdf5, shape (300000, 5000), data_dtype int32
as_csr.h5ad -- CSCDataset: backend hdf5, shape (300000, 5000), data_dtype int32


# 2. Actual abc_atlas_access data

Now it is time for us to work with actual h5ad files distributed with `abc_atlas_access`.

## How are cells linked to h5ad files -- the cell_metadata

The metadata for all of the cells in any given ABC Atlas dataset is in the the `cell_metadata` .csv file.

In [60]:
wmb_metadata = abc_cache.get_metadata_dataframe(
    directory='WMB-10X',
    file_name='cell_metadata'
)

The linkage to h5ad files is encoded in the
- `dataset_label` column, which refers to the `abc_atlas_access` directory where the cell-by-gene data for a cell can be found
- `feature_matrix_label` column, which refers to the file name within that directory

In [61]:
wmb_metadata.sample(n=10, random_state=np.random.default_rng(21231))[['cell_label', 'dataset_label', 'feature_matrix_label']]

,cell_label,dataset_label,feature_matrix_label
1889336,GTTACAGCAGCAATTC-294_A05,WMB-10Xv3,WMB-10Xv3-TH
1960181,AATCACGAGTGGATTA-448_A07,WMB-10Xv3,WMB-10Xv3-Isocortex-2
3813712,GTGCATAGTCTGGTCG-069_C01,WMB-10Xv2,WMB-10Xv2-Isocortex-4
3742959,TTCTTAGAGAACAATC-092_A01,WMB-10Xv2,WMB-10Xv2-OLF
3407039,CGAACATCATTGGTAC-1.05_D01,WMB-10Xv2,WMB-10Xv2-Isocortex-3
3752900,AGGCCACTCTCAACTT-081_C01,WMB-10Xv2,WMB-10Xv2-OLF
3597662,GCTGCTTAGATCGGGT-1.08_B01,WMB-10Xv2,WMB-10Xv2-Isocortex-4
682049,AGATGCTTCACCGGGT-274_A07,WMB-10Xv3,WMB-10Xv3-HPF
2112469,TCATGAGCAGCTGAAG-412_B07,WMB-10Xv3,WMB-10Xv3-HPF
3656614,TGGTTAGAGCCAGGAT-033_B01,WMB-10Xv2,WMB-10Xv2-Isocortex-4


Let's download a small file from the Whole Mouse Brain data release, this one corresponding to
- `dataset_label == WMB-10Xv2`
- `feature_matrix_label == WMB-10Xv2-TH`

**Note:** that you must specify the normalization (`raw` or `log2`) you want when you ask your `abc_cache` to download an h5ad file.

In [62]:
TH_h5ad_path = abc_cache.get_file_path(
    directory='WMB-10Xv2',
    file_name='WMB-10Xv2-TH/log2'
)

WMB-10Xv2-TH-log2.h5ad: 100%|█████████████| 4.04G/4.04G [09:40<00:00, 6.95MMB/s]


Let's subset the cell_metdata to only include cells in this `dataset_label`, `feature_matrix_label` combination

In [63]:
TH_metadata = wmb_metadata[
    np.logical_and(
        wmb_metadata.dataset_label=='WMB-10Xv2',
        wmb_metadata.feature_matrix_label=='WMB-10Xv2-TH'
    )
]

In [64]:
len(TH_metadata)

130555

In [65]:
TH_metadata.head(5)

,cell_label,cell_barcode,barcoded_cell_sample_label,library_label,feature_matrix_label,entity,brain_section_label,library_method,region_of_interest_acronym,donor_label,donor_genotype,donor_sex,dataset_label,x,y,cluster_alias,abc_sample_id
2342830,CAGGTGCAGGCTAGCA-040_C01,CAGGTGCAGGCTAGCA,040_C01,L8TX_180815_01_E08,WMB-10Xv2-TH,cell,NaN,10Xv2,TH,Snap25-IRES2-Cre;Ai14-404124,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2,0.904733,8.810323,1000,da8ad858-d33c-49eb-aae7-df16eba116a0
2343540,TGCGCAGGTTGCGCAC-045_C01,TGCGCAGGTTGCGCAC,045_C01,L8TX_180829_01_C10,WMB-10Xv2-TH,cell,NaN,10Xv2,TH,Snap25-IRES2-Cre;Ai14-407905,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,F,WMB-10Xv2,-1.702466,7.063231,1746,c517bbef-bc4f-4317-8f5c-09ce80605bfe
2343542,CGATGTATCTTGCCGT-042_B01,CGATGTATCTTGCCGT,042_B01,L8TX_180829_01_B09,WMB-10Xv2-TH,cell,NaN,10Xv2,TH,Snap25-IRES2-Cre;Ai14-407901,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2,-10.221148,3.479807,3497,d2a78ca7-cee4-4ec9-9385-54048f6d95c6
2345608,GACTAACGTCCTCTTG-040_B01,GACTAACGTCCTCTTG,040_B01,L8TX_180815_01_D08,WMB-10Xv2-TH,cell,NaN,10Xv2,TH,Snap25-IRES2-Cre;Ai14-404124,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2,-0.080485,7.137248,1039,aa98d92b-fe2b-4c84-8d42-b2c2d0b8b2fb
2345609,GATCGTACAACTGCTA-040_B01,GATCGTACAACTGCTA,040_B01,L8TX_180815_01_D08,WMB-10Xv2-TH,cell,NaN,10Xv2,TH,Snap25-IRES2-Cre;Ai14-404124,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2,-0.154540,7.089330,1039,f29decdc-df2c-4298-853e-c726e5fc226e


Now, let's open the h5ad file we downloaded.

**Note:** it is very important to open the file with `backed = 'r'`. If you do not, anndata will read the entire h5ad file into memory which, depending on the dataset, may be a lot of data that you do not want. Opening the file with `backed = 'r'` tells anndata not to load data into memory until you actually want to do something with it.

In [66]:
TH_h5ad_data = anndata.read_h5ad(
    TH_h5ad_path,
    backed='r'
)

In [67]:
type(TH_h5ad_data)

anndata._core.anndata.AnnData

We have now instantiated an `AnnData` object. `AnnData` objects can contain many member objects for containing data and its related metadata. Consult the official anndata documentation (listed at the top of this notebook) for a full list. The h5ad files distributed with `abc_atlas_access` are very stripped down, they contain only
- `X` the array containing the cell-by-gene data. Each row in this array is a cell. Each column in this array is a gene)
- `obs` a dataframe containing metadata related to every cell in the cell-by-gene dataset (the rows in `X`)
- `var` a dataframe containing metadata related to every gene in the cell-by-gene dataset (the columns in `X`)

Furthermore, because `abc_atlas_access` distributes metadata in many normalized CSV tables, the metadata contained in `obs` and `var` is generally the bare minimum necessary to link the rows and columns in the h5ad file to the other CSV tables in the `abc_atlas_access` dataset.

In [68]:
len(TH_h5ad_data.var)

32285

In [69]:
TH_h5ad_data.var.head(5)

,gene_symbol
gene_identifier,
ENSMUSG00000051951,Xkr4
ENSMUSG00000089699,Gm1992
ENSMUSG00000102331,Gm19938
ENSMUSG00000102343,Gm37381
ENSMUSG00000025900,Rp1


Here we see that `var` just lists the gene identifier and gene symbol of every column in `X`. Note that the `var` dataframe is indexed on the gene identifier, so that you can access individual rows in `var` using the gene identifier, i.e.

In [70]:
TH_h5ad_data.var.loc['ENSMUSG00000089699']

gene_symbol    Gm1992
Name: ENSMUSG00000089699, dtype: object

Similarly, `obs` lists minimal metadata on each cell in `X`, indexed on the `cell_label`

In [71]:
TH_h5ad_data.obs.head(5)

,cell_barcode,library_label,anatomical_division_label
cell_label,,,
CAGGTGCAGGCTAGCA-040_C01,CAGGTGCAGGCTAGCA,L8TX_180815_01_E08,TH
TGCGCAGGTTGCGCAC-045_C01,TGCGCAGGTTGCGCAC,L8TX_180829_01_C10,TH
CGATGTATCTTGCCGT-042_B01,CGATGTATCTTGCCGT,L8TX_180829_01_B09,TH
GACTAACGTCCTCTTG-040_B01,GACTAACGTCCTCTTG,L8TX_180815_01_D08,TH
GATCGTACAACTGCTA-040_B01,GATCGTACAACTGCTA,L8TX_180815_01_D08,TH


These `cell_labels` correspond to the `cell_labels` in `cell_metadata` that were annotated as belonging to this (`dataset_label`, `feature_matrix_label` pair).

**Note:** There are more cells in the h5ad file than in the subset of `cell_metadata`. The "extra" cells in the h5ad file represent cells that failed QC for some reason.

In [72]:
print(f'{len(TH_metadata)} cells in TH_metadata')
print(f'{len(TH_h5ad_data.obs)} cells in this h5ad file')
delta = len(set(TH_metadata.cell_label)-set(TH_h5ad_data.obs.index.values))
print(f'{delta} cells TH_metadata do not appear in this h5ad file')

130555 cells in TH_metadata
131212 cells in this h5ad file
0 cells TH_metadata do not appear in this h5ad file


## Slicing real abc_atlas_access data

With one exception, all of the data distributed by `abc_atlas_access` is stored in CSR format. This file is not that exception.

In [73]:
TH_h5ad_data.X

CSRDataset: backend hdf5, shape (131212, 32285), data_dtype float32

In [74]:
TH_h5ad_data.X.shape

(131212, 32285)

In [75]:
n_cell_gene_pairs = (TH_h5ad_data.X.shape[0]*TH_h5ad_data.X.shape[1])
gb = 4*n_cell_gene_pairs/(1024**3)
print(f"Naively, this data contains {n_cell_gene_pairs} (cell, gene) pairs")
print(f"If each were represented by a 4 byte int/float, that would require {gb} GB")

Naively, this data contains 4236179420 (cell, gene) pairs
If each were represented by a 4 byte int/float, that would require 15.780998095870018 GB


Because the data is in CSR format, whatever manipulation we want to perform should be approached first as a row-slicing operation.

Grabbing a subset of cells can naively be achieved as in our cartoon example above.

In [76]:
TH_cell_idx = np.array([5, 6, 7, 9001, 9002, 9003])

In [77]:
%%time
TH_cell_subset = TH_h5ad_data[TH_cell_idx, :].to_memory()

CPU times: user 3.13 ms, sys: 2.35 ms, total: 5.48 ms
Wall time: 5.46 ms


In [78]:
TH_cell_subset.obs

,cell_barcode,library_label,anatomical_division_label
cell_label,,,
GTGCAGCTCACAGGCC-040_C01,GTGCAGCTCACAGGCC,L8TX_180815_01_E08,TH
CGATCGGTCCTTTCGG-045_A01,CGATCGGTCCTTTCGG,L8TX_180829_01_F09,TH
TGAGCCGTCGGTGTTA-043_A01,TGAGCCGTCGGTGTTA,L8TX_180829_01_C09,TH
AGAATAGAGATCCCAT-037_B01,AGAATAGAGATCCCAT,L8TX_180815_01_F07,TH
CGAGAAGAGTCCATAC-037_B01,CGAGAAGAGTCCATAC,L8TX_180815_01_F07,TH
CGAGAAGTCCCAAGAT-037_B01,CGAGAAGTCCCAAGAT,L8TX_180815_01_F07,TH


We now have a subset of 6 cells drawn from our original dataset. The X matrix is still in sparse format

In [79]:
TH_cell_subset.X

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 20928 stored elements and shape (6, 32285)>

We can get a dense array representation of the `X` data using the `toarray()` function

In [80]:
TH_cell_subset.X.toarray()

array([[8.742955 , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [7.2100267, 0.       , 7.2100267, ..., 0.       , 0.       ,
        0.       ],
       [8.218743 , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [8.806027 , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [7.01022  , 0.       , 0.       , ..., 0.       , 0.       ,
        8.004613 ]], shape=(6, 32285), dtype=float32)

## Slicing real data by genes

To slice the real data by genes, it is most efficient to chunk the data by cells, slice each chunk by genes, and then concatenate the resutling `AnnData` objects into a single object. We will show you how to do this below. First, we draw your attatntion to the `chunked_X` which provides you an iterator over chhunks of rows in an `AnnData` object.

**Note:** This is also a `chunk_X` function which returns a single chunk of the `AnnData` object. Be sure you are calling the correct one.

In [81]:
rows_at_a_time = 10000
row_iterator = TH_h5ad_data.chunked_X(rows_at_a_time)

chunk = next(row_iterator)

The chunks used by this iterator contain
- A chunk of the `X` matrix (in sparse or dense representation depending on the form of the unchunked data)
- The indices of the rows included in the chunk

In [82]:
print(chunk)

(<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 37605474 stored elements and shape (10000, 32285)>, 0, 10000)


As said before, this can be used to assemble, one chunk at a time, a single `AnnData` object that contains a subset of genes from the original `AnnData` object.

Let's select a random subset of 200 genes that we want to subset from our data:

In [83]:
rng = np.random.default_rng(871231)
gene_idx = np.sort(
    rng.choice(
        np.arange(TH_h5ad_data.shape[1]),
        200,
        replace=False
    )
)

In [84]:
%%time

# because we are slicing on genes,
# each chunk in the new AnnData file will have the same
# var dataframe
var = TH_h5ad_data.var.iloc[gene_idx]

rows_at_a_time = 1000
row_iterator = TH_h5ad_data.chunked_X(rows_at_a_time)

# loop over the chunks returned by the row_iterator, creating
# a list of smaller AnnData objects that can later be concatenated
# into one larger AnnData object
sub_adata = []
for chunk in row_iterator:
    obs_chunk = TH_h5ad_data.obs[chunk[1]: chunk[2]]
    x_chunk = chunk[0][:, gene_idx]
    sub_adata.append(
        anndata.AnnData(
            obs=obs_chunk,
            var=var,
            X=x_chunk
        )
    )

# concatenate the smaller AnnData objects into a single object
TH_gene_subset = anndata.concat(sub_adata, merge="same")
del sub_adata
    

CPU times: user 1.29 s, sys: 595 ms, total: 1.88 s
Wall time: 1.89 s


In [85]:
TH_gene_subset.var

,gene_symbol
gene_identifier,
ENSMUSG00000037447,Arid5a
ENSMUSG00000026110,Mgat4a
ENSMUSG00000026043,Col3a1
ENSMUSG00000091283,Gm17234
ENSMUSG00000026027,Stradb
...,...
ENSMUSG00000091736,Yy2
ENSMUSG00000103347,Gm37734
ENSMUSG00000101809,Gm28852


Let's compare this to directly slicing the `AnnData` object all at once.

In [86]:
%%time
alt = TH_h5ad_data[:, gene_idx].to_memory()

CPU times: user 734 ms, sys: 1.3 s, total: 2.03 s
Wall time: 2.17 s


In [87]:
np.testing.assert_array_equal(alt.X.toarray(), TH_gene_subset.X.toarray())

In [88]:
alt.obs.equals(TH_gene_subset.obs)

True

In [89]:
alt.var.equals(TH_gene_subset.var)

True

Directly subsetting the `AnnData` object was actually a littel faster than our "iterate over chunks" strategy and ended up producing equivalent output. However, let us consider the maximum memory footprint of the operation.

In [90]:
read_and_profile.read_by_chunks(
    path=TH_h5ad_path,
    gene_idx=gene_idx,
    rows_at_a_time=rows_at_a_time
)

slicing WMB-10Xv2-TH-log2.h5ad along axis 'gene'
took 1.91e+00 s
maximum memory footprint 7.90e-01 GB (file size is 3.76e+00 GB)


In [91]:
read_and_profile.read(
    path=TH_h5ad_path,
    axis='gene',
    idx=gene_idx
)

slicing WMB-10Xv2-TH-log2.h5ad along 'gene'
took 1.87e+03 ms
maximum memory footprint 3.99e+00 GB (file size is 3.76e+00 GB)


Again: directly subsetting a CSR `AnnData` object by genes required reading the entire file into memory. That was not a problem in this case. We intentionally downloaded one of the smallest h5ad files in `abc_atlas_access`. Some of the h5ad files distributed by `abc_atlas_access`, however, can be several tens of GB in size. Reading these into memory could be prohibitive. Intelligently chunking over the data to achieve your analysis goals will save you some unfortunately timed "Out of Memory" errors.